In [32]:
import os
import pandas as pd
import numpy as np
import copy
from sklearn import preprocessing


## 读取数据

In [47]:
def load_data(proportion=0.1):
    dir_name = "train_set"
    file_list = os.listdir(dir_name)
    l = len(file_list)
    n = int(proportion * l)
    data_list = []
    for i in range(n):
        path = os.path.join(dir_name, file_list[i])
        data_list.append(pd.read_csv(path))
    return pd.concat(data_list, ignore_index=True)

df_raw = load_data(proportion=0.5)

In [3]:
df_raw.head(10)

,Cell Index,Cell X,Cell Y,Height,Azimuth,Electrical Downtilt,Mechanical Downtilt,Frequency Band,RS Power,Cell Altitude,Cell Building Height,Cell Clutter Index,X,Y,Altitude,Building Height,Clutter Index,RSRP
0,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424050,3376785,496,0,15,-102.20
1,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424055,3376790,496,0,15,-100.75
2,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424545,3376390,504,0,5,-70.20
3,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424565,3376410,504,0,5,-65.25
4,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424560,3376405,504,0,5,-64.20
5,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424520,3376365,504,0,5,-75.33
6,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424570,3376415,504,0,5,-67.44
7,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424495,3376340,505,0,5,-78.57
8,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424505,3376350,504,0,5,-72.63
9,1001701,424515.0,3376325.0,24,300,6,3,2585.0,16.2,504,0,5,424485,3376330,505,0,5,-75.25


## 特征提取

In [48]:
df_features = pd.DataFrame()
epsilon = 1e-20
# 发射机功率
df_features["RS Power"] = df_raw["RS Power"]
# 发射机中心频率
df_features["log frequency band"] = np.log(df_raw["Frequency Band"] + epsilon)
# 发射机相对高度
tmp_h = df_raw["Height"] + df_raw["Cell Altitude"] - df_raw["Altitude"]
tmp_h[tmp_h < 0] = 0
df_features["log hb"] = np.log(tmp_h + epsilon)
# 链路距离
tmp_d = np.sqrt(np.square(df_raw["Cell X"] - df_raw["X"]) + np.square(df_raw["Cell Y"] - df_raw["Y"]))
df_features["log d"] = np.log(tmp_d + epsilon)
# 发射机相对高度 * 链路距离
df_features["log hb * log d"] = df_features["log hb"] * df_features["log d"]
# 栅格与信号线相对高度
sum_downtilt_rad = np.deg2rad(df_raw["Electrical Downtilt"] + df_raw["Mechanical Downtilt"]) # 转成弧度
df_features["log delta_hv"] = np.log(np.abs(tmp_h - np.multiply(np.tan(sum_downtilt_rad), tmp_d)) + epsilon) # 往上往下是一样的
# cos(接收位置与信号线的夹角2d)
df_features["cos diff angle 2d"] = np.cos(
    np.arctan((df_raw['X'] - df_raw['Cell X']) / (df_raw['Y'] - df_raw['Cell Y'] + epsilon))
    - np.deg2rad(df_raw['Azimuth']))
# sin(接收位置与信号线的夹角2d)
df_features["sin diff angle 2d"] = np.sin(
    np.arctan((df_raw['X'] - df_raw['Cell X']) / (df_raw['Y'] - df_raw['Cell Y'] + epsilon))
    - np.deg2rad(df_raw['Azimuth']))

# 信号线向量
tmp_vec1 = np.concatenate([
    np.multiply(np.cos(np.array(sum_downtilt_rad)), np.array(df_features["sin diff angle 2d"])).reshape(-1, 1),
    np.multiply(np.cos(np.array(sum_downtilt_rad)), np.array(df_features["cos diff angle 2d"])).reshape(-1, 1),
    -np.sin(np.array(sum_downtilt_rad)).reshape(-1, 1)],
    axis=1)
# 接收处与发射台向量
tmp_vec2 = np.concatenate([
    np.array(df_raw['X'] - df_raw['Cell X']).reshape(-1, 1),
    np.array(df_raw['Y'] - df_raw['Cell Y']).reshape(-1, 1),
    -np.array(tmp_h).reshape(-1, 1)],
    axis=1)

# cos(接收位置与信号线的夹角3d)
df_features["cos diff angle 3d"] = np.sum(np.multiply(tmp_vec1, tmp_vec2), axis=1)/(np.linalg.norm(tmp_vec2, axis=1) + epsilon)
# sin(接收位置与信号线的夹角3d)
tmp_sin_square = 1-np.square(df_features["cos diff angle 3d"])
tmp_sin_square[tmp_sin_square<0] = 0
df_features["sin diff angle 3d"] = np.sqrt(tmp_sin_square)

# 发射处遮挡建筑物高度
df_features["log Cell Building Height"] = np.log(df_raw["Cell Building Height"] + epsilon)
# 接收处遮挡建筑物高度
df_features["log Building Height"] = np.log(df_raw["Building Height"] + epsilon)

# 建筑物覆盖密度（接收处数量/面积）
cell_indices = np.unique(df_raw["Cell Index"])
l = df_raw.shape[0]
density = np.zeros(l)
for cell_index in cell_indices:
    tmp_cell = df_raw[df_raw["Cell Index"]==cell_index]
    num_receivers = tmp_cell.shape[0]
    surface = (np.max(tmp_cell["X"]) - np.min(tmp_cell["X"]))\
            * (np.max(tmp_cell["Y"]) - np.min(tmp_cell["Y"]))
    density[df_raw["Cell Index"]==cell_index] = num_receivers/surface
    
df_features["density"] = density

# 发射处/接收处地物类型
onehot_encoder = preprocessing.OneHotEncoder()
categorical_features = np.concatenate([np.array(df_raw["Cell Clutter Index"]).reshape(-1, 1),
                                       np.array(df_raw["Clutter Index"]).reshape(-1, 1)], axis=1)

onehot_encoder = preprocessing.OneHotEncoder(categories=[list(range(1, 21)), list(range(1, 21))])
onehot_categorical_features = onehot_encoder.fit_transform(categorical_features).toarray()

# RSPS(目标)
df_features["RSRP"] = df_raw["RSRP"]

In [5]:
df_features.head(10)

,RS Power,log frequency band,log hb,log d,log hb * log d,log delta_hv,cos diff angle 2d,sin diff angle 2d,cos diff angle 3d,sin diff angle 3d,log Cell Building Height,log Building Height,density,RSRP
0,16.2,7.857481,3.465736,6.483235,22.469179,4.271048,0.967311,0.253594,0.500898,0.865506,-46.051702,-46.051702,0.000747,-102.20
1,16.2,7.857481,3.465736,6.483235,22.469179,4.271048,0.964513,0.264036,0.500898,0.865506,-46.051702,-46.051702,0.000747,-100.75
2,16.2,7.857481,3.178054,4.270943,13.573286,2.538558,0.091065,0.995845,0.517957,0.855407,-46.051702,-46.051702,0.000747,-70.20
3,16.2,7.857481,3.178054,4.591228,14.591168,2.125950,-0.008125,0.999967,0.516830,0.856088,-46.051702,-46.051702,0.000747,-65.25
4,16.2,7.857481,3.178054,4.519479,14.363149,2.247309,0.011209,0.999937,0.517355,0.855771,-46.051702,-46.051702,0.000747,-64.20
5,16.2,7.857481,3.178054,3.696632,11.748094,2.868769,0.388722,0.921355,0.504359,0.863494,-46.051702,-46.051702,0.000747,-75.33
6,16.2,7.857481,3.178054,4.658475,14.804884,1.987105,-0.024948,0.999689,0.516244,0.856442,-46.051702,-46.051702,0.000747,-67.44
7,16.2,7.857481,3.135494,3.218876,10.092767,2.946562,0.992820,0.119615,0.469350,0.883012,-46.051702,-46.051702,0.000747,-78.57
8,16.2,7.857481,3.178054,3.293086,10.465604,2.982412,0.785872,0.618389,0.472744,0.881200,-46.051702,-46.051702,0.000747,-72.63
9,16.2,7.857481,3.135494,3.414897,10.707389,2.900483,0.936442,-0.350823,0.488251,0.872703,-46.051702,-46.051702,0.000747,-75.25


In [31]:
df_features.shape

(5957548, 14)

## 训练模型


In [35]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeRegressor

In [49]:
X = np.array(df_features)[:, :-1]
continous_feature_len = X.shape[1]
X = np.concatenate([X, onehot_categorical_features], axis=1)
y = np.array(df_features)[:, -1]

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [52]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
X_train_scaled = X_train
X_test_scaled = X_test
X_train_scaled[:, :continous_feature_len] = scaler.fit_transform(X_train[:, :continous_feature_len])
X_test_scaled[:, :continous_feature_len] = scaler.transform(X_test[:, :continous_feature_len])
# model = linear_model.LinearRegression()
model = DecisionTreeRegressor(max_depth=40)
model.fit(X_train_scaled, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=40, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [28]:
scaler.mean_



array([ 1.25375642e+01,  7.85780485e+00,  2.15749835e+00,  5.48716963e+00,
        1.20915046e+01,  2.78705480e+00,  2.20500199e-02,  2.54606415e-02,
        2.28527298e-01,  6.68035068e-01, -3.13760914e+01, -3.89782232e+01,
        1.00143029e-03])

In [53]:
y_test_pred = model.predict(X_test)
y_test_binary = copy.copy(y_test)
y_test_pred_binary = copy.copy(y_test_pred)
y_test_binary[y_test>=-103] = 0
y_test_binary[y_test<-103] = 1
y_test_pred_binary[y_test_pred>=-103] = 0
y_test_pred_binary[y_test_pred<-103] = 1

In [54]:
print ("mean squared error: {}".format(mean_squared_error(y_test, y_test_pred)))
print ("precision : {}".format(precision_score(y_test_binary, y_test_pred_binary)))
print ("recall : {}".format(recall_score(y_test_binary, y_test_pred_binary)))
print ("f1 score: {}".format(f1_score(y_test_binary, y_test_pred_binary)))

mean squared error: 44.285528462894824
precision : 0.6180445874243957
recall : 0.6123464446551743
f1 score: 0.6151823215226876


In [43]:
import tensorflow as tf

In [42]:
y_train_binary = copy.copy(y_train)
y_train_binary[y_train>=-103] = 0
y_train_binary[y_train<-103] = 1

In [24]:
# feature_len = X_test_scaled.shape[1]
# num_hidden_units_layer_1 = 64
# num_hidden_units_layer_2 = 64

# X = tf.placeholder("float", [None, feature_len])
# y = tf.placeholder("float", [None, 1])
# y_binary = tf.placeholder("float", [None, 1])

# W1 = tf.Variable(tf.random_normal([feature_len, num_hidden_units_layer_1]))
# b1 = tf.Variable(tf.zeros([1, num_hidden_units_layer_1]) + 0.1)
# Z1 = tf.matmul(X, W1) + b1
# A1 = tf.nn.relu(Z1)

# W2 = tf.Variable(tf.random_normal([num_hidden_units_layer_1, num_hidden_units_layer_2]))
# b2 = tf.Variable(tf.zeros([1, num_hidden_units_layer_2]) + 0.1)
# Z2 = tf.matmul(A1, W2) + b2
# A2 = tf.nn.relu(Z2)
# A2 = tf.nn.sigmoid(Z2)

# W3 = tf.Variable(tf.random_normal([num_hidden_units_layer_2, 1]))
# b3 = tf.Variable(tf.zeros([1, 1]) + 0.1)
# Z3 = tf.matmul(A2, W3) + b3
# A3 = Z3

# mse_loss = tf.reduce_mean((A3 - y) ** 2)
# op1 = tf.train.AdamOptimizer().minimize(mse_loss)


# threshold = tf.constant(-103.0)
# output_prob = tf.nn.sigmoid(threshold - A3)
# cross_entropy = -tf.reduce_mean(y_binary * tf.log(tf.clip_by_value(output_prob, 1e-10, 1.0)))
# op2 = tf.train.AdamOptimizer().minimize(cross_entropy)


# n_epoch = 3000

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
    
#     for i in range(n_epoch):
#         _, train_mse_loss = sess.run([op1, mse_loss], feed_dict={X: X_train_scaled, y: y_train.reshape(-1, 1)})#, y_binary: y_train_binary.reshape(-1, 1)})
#         test_mse_loss, y_test_pred = sess.run([mse_loss, A3], feed_dict={X: X_test_scaled, y: y_test.reshape(-1, 1)})#, y_binary: y_test_binary.reshape(-1, 1)})

#         _, train_cross_entropy_loss = sess.run([op2, cross_entropy], feed_dict={X: X_train_scaled, y: y_train.reshape(-1, 1), y_binary: y_train_binary.reshape(-1, 1)})
#         test_cross_entropy_loss = sess.run(cross_entropy, feed_dict={X: X_test_scaled, y: y_test.reshape(-1, 1), y_binary: y_test_binary.reshape(-1, 1)})
        
#         y_test_pred_binary = copy.copy(y_test_pred)
#         y_test_pred_binary[y_test_pred>=-103] = 0
#         y_test_pred_binary[y_test_pred<-103] = 1
#         test_f1_score = f1_score(y_test_binary, y_test_pred_binary)
        
#         print ("train mse loss: {}, test mse loss: {}, test f1 score: {}".format(train_mse_loss, test_mse_loss, test_f1_score))
        
#         print ("train cross entropy loss: {}, test cross entropy loss: {}".format(train_cross_entropy_loss, test_cross_entropy_loss))
        

In [25]:
# from tensorflow.python.framework import graph_util
# pb_file_path = os.getcwd()
# with tf.Session(graph=tf.Graph()) as sess:
    
#     feature_len = X_test_scaled.shape[1]
#     num_hidden_units_layer_1 = 64
#     num_hidden_units_layer_2 = 64

#     X = tf.placeholder("float", [None, feature_len])
#     y = tf.placeholder("float", [None, 1])
#     y_binary = tf.placeholder("float", [None, 1])

#     W1 = tf.Variable(tf.random_normal([feature_len, num_hidden_units_layer_1]))
#     b1 = tf.Variable(tf.zeros([1, num_hidden_units_layer_1]) + 0.1)
#     Z1 = tf.matmul(X, W1) + b1
#     A1 = tf.nn.relu(Z1)

#     W2 = tf.Variable(tf.random_normal([num_hidden_units_layer_1, num_hidden_units_layer_2]))
#     b2 = tf.Variable(tf.zeros([1, num_hidden_units_layer_2]) + 0.1)
#     Z2 = tf.matmul(A1, W2) + b2
#     A2 = tf.nn.sigmoid(Z2)

#     W3 = tf.Variable(tf.random_normal([num_hidden_units_layer_2, 1]))
#     b3 = tf.Variable(tf.zeros([1, 1]) + 0.1)
#     Z3 = tf.add(tf.matmul(A2, W3), b3, name="output")
#     A3 = Z3

#     mse_loss = tf.reduce_mean((A3 - y) ** 2)
#     op1 = tf.train.AdadeltaOptimizer(learning_rate=30).minimize(mse_loss)


#     threshold = tf.constant(-103.0)
#     output_prob = tf.nn.sigmoid(threshold - A3)
#     cross_entropy = -tf.reduce_mean(y_binary * tf.log(tf.clip_by_value(output_prob, 1e-10, 1.0)))
#     op2 = tf.train.AdamOptimizer().minimize(cross_entropy)
    
    
#     sess.run(tf.global_variables_initializer())
#     n_epoch = 10
#     constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['output'])
    
#     for i in range(n_epoch):
#         _, train_mse_loss = sess.run([op1, mse_loss], feed_dict={X: X_train_scaled, y: y_train.reshape(-1, 1), y_binary: y_train_binary.reshape(-1, 1)})
#         test_mse_loss, y_test_pred = sess.run([mse_loss, A3], feed_dict={X: X_test_scaled, y: y_test.reshape(-1, 1), y_binary: y_test_binary.reshape(-1, 1)})

#         y_test_pred_binary = copy.copy(y_test_pred)
#         y_test_pred_binary[y_test_pred>=-103] = 0
#         y_test_pred_binary[y_test_pred<-103] = 1
#         test_f1_score = f1_score(y_test_binary, y_test_pred_binary)
        
#         print ("train mse loss: {}, test mse loss: {}, test f1 score: {}".format(train_mse_loss, test_mse_loss, test_f1_score))
    
    
#     for i in range(n_epoch):
#         _, train_cross_entropy_loss = sess.run([op2, cross_entropy], feed_dict={X: X_train_scaled, y: y_train.reshape(-1, 1), y_binary: y_train_binary.reshape(-1, 1)})
#         test_mse_loss, test_cross_entropy_loss, y_test_pred = sess.run([mse_loss, cross_entropy, A3], feed_dict={X: X_test_scaled, y: y_test.reshape(-1, 1), y_binary: y_test_binary.reshape(-1, 1)})
        
#         y_test_pred_binary = copy.copy(y_test_pred)
#         y_test_pred_binary[y_test_pred>=-103] = 0
#         y_test_pred_binary[y_test_pred<-103] = 1
#         test_f1_score = f1_score(y_test_binary, y_test_pred_binary)
        
#         print ("test mse loss: {}, test cross entropy loss: {}, test f1 score: {}".format(test_mse_loss, test_cross_entropy_loss, test_f1_score))
        
#     with tf.gfile.FastGFile(pb_file_path+'/model.pb', mode='wb') as f:
#         f.write(constant_graph.SerializeToString())

In [55]:
with tf.Session(graph=tf.Graph()) as sess:
    
    feature_len = X_test_scaled.shape[1]
    num_hidden_units_layer_1 = 64
    num_hidden_units_layer_2 = 64

    X = tf.placeholder("float", [None, feature_len], name="myInput")
    y = tf.placeholder("float", [None, 1])
#     y_binary = tf.placeholder("float", [None, 1])

    W1 = tf.Variable(tf.random_normal([feature_len, num_hidden_units_layer_1]))
    b1 = tf.Variable(tf.zeros([1, num_hidden_units_layer_1]) + 0.1)
    Z1 = tf.matmul(X, W1) + b1
    A1 = tf.nn.relu(Z1)

    W2 = tf.Variable(tf.random_normal([num_hidden_units_layer_1, num_hidden_units_layer_2]))
    b2 = tf.Variable(tf.zeros([1, num_hidden_units_layer_2]) + 0.1)
    Z2 = tf.matmul(A1, W2) + b2
    A2 = tf.nn.relu(Z2)
#     A2 = tf.nn.sigmoid(Z2)


    W3 = tf.Variable(tf.random_normal([num_hidden_units_layer_2, 1]))
    b3 = tf.Variable(tf.zeros([1, 1]) + 0.1)
    Z3 = tf.add(tf.matmul(A2, W3), b3)
    A3 = Z3
    tf.identity(A3, name="myOutput")
    

    mse_loss = tf.reduce_mean((A3 - y) ** 2)
    op1 = tf.train.AdadeltaOptimizer(learning_rate=20).minimize(mse_loss)


#     threshold = tf.constant(-103.0)
#     output_prob = tf.nn.sigmoid(threshold - A3)
#     cross_entropy = -tf.reduce_mean(y_binary * tf.log(tf.clip_by_value(output_prob, 1e-10, 1.0)))
#     op2 = tf.train.AdamOptimizer().minimize(cross_entropy)
    
    
    sess.run(tf.global_variables_initializer())
    n_epoch = 1000
    summary_writer = tf.summary.FileWriter('./log/', sess.graph)
    for i in range(n_epoch):
        _, train_mse_loss = sess.run([op1, mse_loss], feed_dict={X: X_train_scaled, y: y_train.reshape(-1, 1)})#, y_binary: y_train_binary.reshape(-1, 1)})
        test_mse_loss, y_test_pred = sess.run([mse_loss, A3], feed_dict={X: X_test_scaled, y: y_test.reshape(-1, 1)})#, y_binary: y_test_binary.reshape(-1, 1)})

        y_test_pred_binary = copy.copy(y_test_pred)
        y_test_pred_binary[y_test_pred>=-103] = 0
        y_test_pred_binary[y_test_pred<-103] = 1
        test_f1_score = f1_score(y_test_binary, y_test_pred_binary)
        
        print ("train mse loss: {}, test mse loss: {}, test f1 score: {}".format(train_mse_loss, test_mse_loss, test_f1_score))
    
    
#     for i in range(5):
#         _, train_cross_entropy_loss = sess.run([op2, cross_entropy], feed_dict={X: X_train_scaled, y: y_train.reshape(-1, 1), y_binary: y_train_binary.reshape(-1, 1)})
#         test_mse_loss, test_cross_entropy_loss, y_test_pred = sess.run([mse_loss, cross_entropy, A3], feed_dict={X: X_test_scaled, y: y_test.reshape(-1, 1), y_binary: y_test_binary.reshape(-1, 1)})
        
#         y_test_pred_binary = copy.copy(y_test_pred)
#         y_test_pred_binary[y_test_pred>=-103] = 0
#         y_test_pred_binary[y_test_pred<-103] = 1
#         test_f1_score = f1_score(y_test_binary, y_test_pred_binary)
        
#         print ("test mse loss: {}, test cross entropy loss: {}, test f1 score: {}".format(test_mse_loss, test_cross_entropy_loss, test_f1_score))
        
    tf.saved_model.simple_save(sess, "./model/", inputs={"myInput": X }, outputs={"myOutput": A3})

train mse loss: 9578.6806640625, test mse loss: 7346.40673828125, test f1 score: 0.2262572243359135
train mse loss: 7328.66259765625, test mse loss: 5737.25634765625, test f1 score: 0.22816616176979201
train mse loss: 5722.0029296875, test mse loss: 4559.77392578125, test f1 score: 0.22337617841828297
train mse loss: 4547.71240234375, test mse loss: 3709.083984375, test f1 score: 0.22596486303292548
train mse loss: 3698.158935546875, test mse loss: 3081.96240234375, test f1 score: 0.2197266779968803
train mse loss: 3072.845703125, test mse loss: 2620.433837890625, test f1 score: 0.2241894549921296
train mse loss: 2611.704833984375, test mse loss: 2274.58203125, test f1 score: 0.21574032414293098
train mse loss: 2267.00390625, test mse loss: 2016.0076904296875, test f1 score: 0.22313885203019457
train mse loss: 2008.33203125, test mse loss: 1819.9495849609375, test f1 score: 0.2101744845962082
train mse loss: 1813.2119140625, test mse loss: 1671.728515625, test f1 score: 0.2217565105400

train mse loss: 518.9132080078125, test mse loss: 518.0658569335938, test f1 score: 0.14923956899707363
train mse loss: 516.7548217773438, test mse loss: 512.9744873046875, test f1 score: 0.23521813549332699
train mse loss: 512.175537109375, test mse loss: 511.2935485839844, test f1 score: 0.14823089098463363
train mse loss: 510.01043701171875, test mse loss: 506.1645812988281, test f1 score: 0.23562657818211652
train mse loss: 505.4034118652344, test mse loss: 504.5299072265625, test f1 score: 0.1475151573017616
train mse loss: 503.27398681640625, test mse loss: 499.4068603515625, test f1 score: 0.23596873363178617
train mse loss: 498.67919921875, test mse loss: 497.83392333984375, test f1 score: 0.14647696420279369
train mse loss: 496.6057434082031, test mse loss: 492.76348876953125, test f1 score: 0.23641830426031685
train mse loss: 492.0677185058594, test mse loss: 491.2736511230469, test f1 score: 0.14546366147098416
train mse loss: 490.0742492675781, test mse loss: 486.2801818847

train mse loss: 345.77471923828125, test mse loss: 343.2320556640625, test f1 score: 0.24871999002465067
train mse loss: 342.9267578125, test mse loss: 343.8949279785156, test f1 score: 0.1136531632352409
train mse loss: 343.2218933105469, test mse loss: 340.69140625, test f1 score: 0.24889985036812518
train mse loss: 340.38885498046875, test mse loss: 341.4009704589844, test f1 score: 0.11297035281166674
train mse loss: 340.7342224121094, test mse loss: 338.2217102050781, test f1 score: 0.24911728053170867
train mse loss: 337.92205810546875, test mse loss: 338.95928955078125, test f1 score: 0.11225762025739071
train mse loss: 338.3000183105469, test mse loss: 335.7852783203125, test f1 score: 0.24949170555889283
train mse loss: 335.48736572265625, test mse loss: 336.54315185546875, test f1 score: 0.11164219154533922
train mse loss: 335.8908386230469, test mse loss: 333.3974304199219, test f1 score: 0.24977391968315976
train mse loss: 333.1014099121094, test mse loss: 334.145263671875,

train mse loss: 266.06353759765625, test mse loss: 263.9873046875, test f1 score: 0.2562516314547572
train mse loss: 263.71844482421875, test mse loss: 265.0746765136719, test f1 score: 0.08476296673325703
train mse loss: 264.656494140625, test mse loss: 262.5874328613281, test f1 score: 0.25645515037384164
train mse loss: 262.3194580078125, test mse loss: 263.67938232421875, test f1 score: 0.0839588168849592
train mse loss: 263.266357421875, test mse loss: 261.1969299316406, test f1 score: 0.2565397873728015
train mse loss: 260.929443359375, test mse loss: 262.2864074707031, test f1 score: 0.08309875813128327
train mse loss: 261.87841796875, test mse loss: 259.81427001953125, test f1 score: 0.2566525002410608
train mse loss: 259.5471496582031, test mse loss: 260.895263671875, test f1 score: 0.08233128725255118
train mse loss: 260.49127197265625, test mse loss: 258.4295654296875, test f1 score: 0.2567520554474941
train mse loss: 258.1634826660156, test mse loss: 259.5072021484375, test

train mse loss: 218.4844512939453, test mse loss: 220.01466369628906, test f1 score: 0.05908547775182324
train mse loss: 219.62704467773438, test mse loss: 217.83055114746094, test f1 score: 0.26291911500914716
train mse loss: 217.585205078125, test mse loss: 219.12570190429688, test f1 score: 0.058820355765721456
train mse loss: 218.7398223876953, test mse loss: 216.9486846923828, test f1 score: 0.2630557084497717
train mse loss: 216.702880859375, test mse loss: 218.25538635253906, test f1 score: 0.05859267003727584
train mse loss: 217.87081909179688, test mse loss: 216.0808868408203, test f1 score: 0.26320311643444816
train mse loss: 215.83419799804688, test mse loss: 217.3932342529297, test f1 score: 0.058141237434986594
train mse loss: 217.0103759765625, test mse loss: 215.2267608642578, test f1 score: 0.26334510796149463
train mse loss: 214.9785919189453, test mse loss: 216.54249572753906, test f1 score: 0.05765993265993266
train mse loss: 216.1614532470703, test mse loss: 214.385

...

train mse loss: 172.67330932617188, test mse loss: 174.53619384765625, test f1 score: 0.030178270139666046
train mse loss: 174.29507446289062, test mse loss: 172.48812866210938, test f1 score: 0.2727741700940729
train mse loss: 172.30148315429688, test mse loss: 174.17269897460938, test f1 score: 0.03001092364747095
train mse loss: 173.93096923828125, test mse loss: 172.13076782226562, test f1 score: 0.2728333363248707
train mse loss: 171.94613647460938, test mse loss: 173.8280792236328, test f1 score: 0.029793882249753312
train mse loss: 173.58506774902344, test mse loss: 171.79026794433594, test f1 score: 0.2729228228471187
train mse loss: 171.607666015625, test mse loss: 173.4956512451172, test f1 score: 0.029542229139100085
train mse loss: 173.25135803222656, test mse loss: 171.45809936523438, test f1 score: 0.27319978705292036
train mse loss: 171.277099609375, test mse loss: 173.16949462890625, test f1 score: 0.029226826933961777
train mse loss: 172.924072265625, test mse loss: 17

train mse loss: 161.2657928466797, test mse loss: 159.4492645263672, test f1 score: 0.27577915371037287
train mse loss: 159.30726623535156, test mse loss: 161.2871856689453, test f1 score: 0.022876771385261518
train mse loss: 161.05172729492188, test mse loss: 159.22579956054688, test f1 score: 0.27589328570783894
train mse loss: 159.08399963378906, test mse loss: 161.0676727294922, test f1 score: 0.022728142337861107
train mse loss: 160.83267211914062, test mse loss: 159.00021362304688, test f1 score: 0.27603946354331366
train mse loss: 158.85841369628906, test mse loss: 160.84103393554688, test f1 score: 0.022591634970403292
train mse loss: 160.60646057128906, test mse loss: 158.76614379882812, test f1 score: 0.27621865402983886
train mse loss: 158.62428283691406, test mse loss: 160.6016845703125, test f1 score: 0.022425833223095614
train mse loss: 160.36721801757812, test mse loss: 158.5098114013672, test f1 score: 0.2762911268390273
train mse loss: 158.36781311035156, test mse loss

train mse loss: 149.353759765625, test mse loss: 147.61378479003906, test f1 score: 0.27672015827654434
train mse loss: 147.46133422851562, test mse loss: 149.24232482910156, test f1 score: 0.018116743762929866
train mse loss: 149.0254669189453, test mse loss: 147.29859924316406, test f1 score: 0.2766003072683957
train mse loss: 147.14642333984375, test mse loss: 148.93934631347656, test f1 score: 0.01805017372150458
train mse loss: 148.72254943847656, test mse loss: 147.0063934326172, test f1 score: 0.27652785177902267
train mse loss: 146.85458374023438, test mse loss: 148.6632537841797, test f1 score: 0.017878665425092678
train mse loss: 148.44680786132812, test mse loss: 146.7299041748047, test f1 score: 0.27650248078205103
train mse loss: 146.57850646972656, test mse loss: 148.4131622314453, test f1 score: 0.01772760501602654
train mse loss: 148.19715881347656, test mse loss: 146.49026489257812, test f1 score: 0.2766691295995271
train mse loss: 146.33895874023438, test mse loss: 14

train mse loss: 141.3105926513672, test mse loss: 139.5596923828125, test f1 score: 0.27667162421763647
train mse loss: 139.42227172851562, test mse loss: 141.3666229248047, test f1 score: 0.014619138552938681
train mse loss: 141.15443420410156, test mse loss: 139.3968505859375, test f1 score: 0.2767771571069225
train mse loss: 139.25987243652344, test mse loss: 141.20327758789062, test f1 score: 0.014564926750939046
train mse loss: 140.9914093017578, test mse loss: 139.23313903808594, test f1 score: 0.276869004724033
train mse loss: 139.09649658203125, test mse loss: 141.0392608642578, test f1 score: 0.014488754943367928
train mse loss: 140.82766723632812, test mse loss: 139.06932067871094, test f1 score: 0.2769285289075335
train mse loss: 138.9332275390625, test mse loss: 140.87814331054688, test f1 score: 0.014357384960480044
train mse loss: 140.66673278808594, test mse loss: 138.90733337402344, test f1 score: 0.2771114897722587
train mse loss: 138.7718505859375, test mse loss: 140.

KeyboardInterrupt: 

In [56]:
print ("mean squared error: {}".format(mean_squared_error(y_test, y_test_pred)))
print ("precision : {}".format(precision_score(y_test_binary, y_test_pred_binary)))
print ("recall : {}".format(recall_score(y_test_binary, y_test_pred_binary)))
print ("f1 score: {}".format(f1_score(y_test_binary, y_test_pred_binary)))

mean squared error: 138.0754355711452
precision : 0.24500736833083114
recall : 0.31951410347475867
f1 score: 0.2773439578358314


In [ ]:
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, ["serve"], "./model")
    graph = tf.get_default_graph()
    
    X = sess.graph.get_tensor_by_name('myInput:0')
    y = sess.graph.get_tensor_by_name('myOutput:0')

    y_test_pred = sess.run(y, feed_dict={X: X_test})